#  Natural Language Processing Case Study

## Sentiment Classification for Customer reviews

## 1. Import required packages

In [34]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.feature_extraction import text

# import sklearn classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## 2. Load dataset

In [36]:
data = pd.read_csv('data/IMDB Dataset.csv')

In [37]:
data = data.sample(frac=.5)

In [38]:
data.head()

,review,sentiment
21560,A hilarious Action comedy in which Damian Szif...,positive
48917,The old axiom that bored people are boring peo...,negative
40417,"First of all, I have to start this comment by ...",positive
33849,"If I didn't know any better, I would have thou...",negative
45148,"The Broadway musical, ""A Chorus Line"" is argua...",negative


In [39]:
data.shape

(25000, 2)

In [40]:
i = 0
print('Review =\n{}\n'.format(data.iloc[i,0]))
print("Sentiment = {}".format(data.iloc[i,1]))

Review =
A hilarious Action comedy in which Damian Szifron takes into the lives of Díaz, a cop whose wife has cheated on him. He is living in a hotel feeling guilty about his wife's unfaithfulness, falling into depression he stops caring, running red lights just for fun, feeling sorry for himself. And Silberman, a Jewish shrink on probation, a leftist liberal who does not trust cops at all. He is told to accompany Díaz in his daily duties, unable to refuse due to the terms of his probation. Soon the situation reverts when Silberman himself finds out his wife is cheating on him, and ends up being comforted by the very person he was supposed to help, a person he did not trust at all in the beginning. A nuclear conspiracy, car thieves, international spies, a hysterical wife. Weird characters in a delightful comedy about friendship and heroism.

Sentiment = positive


##  3. Analyze Dataset

### Apply tokenizing

_______________

### <span style="color:blue">**TODO: Experiment with both Vectorizer choices and different parameters!**</span>

In [41]:
# set stopwords
stopwords = text.ENGLISH_STOP_WORDS

# initialize and fit vectorizer
vect = CountVectorizer(max_features=3000, stop_words=stopwords, token_pattern=r'\b[^\d\W]+\b')\
                      .fit(data['review'])
#vect = TfidfVectorizer(max_features=3000, stop_words=stopwords.words('english'), token_pattern=r'\b[^\d\W]+\b')\
#                      .fit(data['review'])

# apply vectorizer to data set
X = vect.transform(data['review'])

Number of discarded tokens: since we chose to use only the most frequent tokens, the number of stop words goes up a lot!

In [42]:
print(len(vect.stop_words_))

73158


In [43]:
y = data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

Basic information on tokens and size of dictionary

In [44]:
feature_names = vect.get_feature_names()

print("Number of features: {}\n".format(len(feature_names)))
print("First 100 features:\n{}\n".format(feature_names[:100]))
print("Features 110 to 130:\n{}\n".format(feature_names[110:130]))
print("Every 100th feature:\n{}\n".format(feature_names[::100]))

Number of features: 2000

First 100 features:
['ability', 'able', 'absolute', 'absolutely', 'absurd', 'academy', 'accent', 'accept', 'accident', 'according', 'accurate', 'act', 'acted', 'acting', 'action', 'actions', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'adam', 'adaptation', 'add', 'added', 'addition', 'adds', 'admit', 'adult', 'adults', 'adventure', 'advice', 'affair', 'afraid', 'african', 'age', 'aged', 'agent', 'ages', 'ago', 'agree', 'ahead', 'air', 'al', 'alan', 'albert', 'alien', 'aliens', 'alive', 'allen', 'allow', 'allowed', 'allows', 'amateurish', 'amazing', 'america', 'american', 'americans', 'amusing', 'ancient', 'anderson', 'andy', 'angles', 'angry', 'animal', 'animals', 'animated', 'animation', 'ann', 'anna', 'anne', 'annoying', 'answer', 'anthony', 'anti', 'anybody', 'anymore', 'apart', 'apartment', 'apparent', 'apparently', 'appeal', 'appealing', 'appear', 'appearance', 'appeared', 'appears', 'appreciate', 'approach', 'appropriate', 'a

## 4. Split into train and test set

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. Choose classifier method and fit on data
_______________
### <span style="color:blue">**TODO: Experiment with different classifier choices and different parameters!**</span>

In [56]:
clf = DecisionTreeClassifier(max_depth=15, min_samples_leaf=10)
#clf = LogisticRegression(max_iter=5000)
#clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
#clf = GaussianNB()
#clf = MLPClassifier(alpha=.01, max_iter=500)

In [57]:
#clf = clf.fit(X_train, y_train)
clf = clf.fit(X_train.toarray(), y_train)

## 6. Use trained model to predict labels for train and test set

In [58]:
y_pred = clf.predict(X_test.toarray())
y_train_pred = clf.predict(X_train.toarray())

In [59]:
test_accuracy = accuracy_score(y_test, y_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)

In [60]:
print("accuracy on training set = {:1.1f}%".format(100*train_accuracy))
print("accuracy on test set\t = {:1.1f}%".format(100*test_accuracy))

accuracy on training set = 77.6%
accuracy on test set	 = 73.4%


## Aside: Look at tokens associated with positive or negative sentiments

In [26]:
try:
    indices = np.argsort(clf.coef_[0])
    feature_names = np.array(vect.get_feature_names())[indices]
    neg_unigrams = feature_names[:100]
    print('Words associated with negative reviews:')
    print(neg_unigrams)
    pos_unigrams = feature_names[-100:-1]
    print('\n')
    print('Words associated with positive reviews:')
    print(pos_unigrams)
except:
    print('Print classifier does not have coeff attribute')

Print classifier does not have coeff attribute


## Apply sentiment classification to new sentence

In [27]:
sentiment = lambda x: 'positive' if x == 1 else 'negative'

### <span style="color:blue">**TODO: Write your own review and check the detected sentiment!**</span>

In [28]:
my_review = ['this movie was shit and crap at the same time']

In [29]:
# vectorize new review
x_rev = vect.transform(my_review)
# Classify new review
y_pred = clf.predict(x_rev.toarray())
print('Sentiment of review: {}'.format(sentiment(y_pred)))

Sentiment of review: positive


In [30]:
import pickle

In [31]:
pickle.dump(clf, open('nlp_model.pkl','wb'))

In [32]:
pickle.dump(vect, open('nlp_vect.pkl','wb'))